In [16]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("ss.pdf")
docs = loader.load()
docs

[Document(metadata={'producer': 'Foxit PDF Creator Version 7.3.0.1225', 'creator': 'Foxit Software Inc.', 'creationdate': '2016-05-06T20:31:42+05:30', 'moddate': '2017-10-25T10:28:18-04:00', 'spdf': '1132', 'subject': '', 'title': '', 'author': 'Dell', 'keywords': '', 'enhanced': 'By PDF Enhancer 3.8.10571/Unix', 'source': 'ss.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='Solar System – GK Notes in PDF \nOur little corner of the Universe is called the solar system. Questions on the Solar System funnily turn up \nin both Geography and Science. You can find a fe w questions on the Solar System in many competitive \nrecruitment exams like Railways RRB, SSC CGL, SBI PO, SBI Clerk, IBPS Clerk, and IBPS PO. So understand \nthe basics of the Solar System in these GK Notes. You can download this Solar System article for revision \nlater.  \nThe Solar System \nThe solar system consists of the Sun and its eight main planets, their satellites, asteroids, comets, \nmeteors a

In [17]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    
)
document = text_splitter.split_documents(docs)
document[:5]


[Document(metadata={'producer': 'Foxit PDF Creator Version 7.3.0.1225', 'creator': 'Foxit Software Inc.', 'creationdate': '2016-05-06T20:31:42+05:30', 'moddate': '2017-10-25T10:28:18-04:00', 'spdf': '1132', 'subject': '', 'title': '', 'author': 'Dell', 'keywords': '', 'enhanced': 'By PDF Enhancer 3.8.10571/Unix', 'source': 'ss.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='Solar System – GK Notes in PDF \nOur little corner of the Universe is called the solar system. Questions on the Solar System funnily turn up \nin both Geography and Science. You can find a fe w questions on the Solar System in many competitive \nrecruitment exams like Railways RRB, SSC CGL, SBI PO, SBI Clerk, IBPS Clerk, and IBPS PO. So understand \nthe basics of the Solar System in these GK Notes. You can download this Solar System article for revision \nlater.  \nThe Solar System \nThe solar system consists of the Sun and its eight main planets, their satellites, asteroids, comets, \nmeteors a

In [18]:
##fiass vector db
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings 
db = FAISS.from_documents(
    document,
    embedding=OllamaEmbeddings()
)

In [19]:
db

In [20]:
query ="what is mars"
result = db.similarity_search(query)
result

[Document(id='20ed6c4b-0b85-4b24-bd46-1e1c8b012509', metadata={'producer': 'Foxit PDF Creator Version 7.3.0.1225', 'creator': 'Foxit Software Inc.', 'creationdate': '2016-05-06T20:31:42+05:30', 'moddate': '2017-10-25T10:28:18-04:00', 'spdf': '1132', 'subject': '', 'title': '', 'author': 'Dell', 'keywords': '', 'enhanced': 'By PDF Enhancer 3.8.10571/Unix', 'source': 'ss.pdf', 'total_pages': 7, 'page': 3, 'page_label': '4'}, page_content='Mercury \n\uf0b7 Closest planet to the Sun.  \n\uf0b7 Orbits the Sun the quickest. \n\uf0b7 No significant atmosphere. \n\uf0b7 Rotation period is longer than its orbital (revolution) period.  \n\uf0b7 No natural satellites (moons). \n\uf0b7 Spacecrafts to Mercury: Mariner 10, MESSENGER, BepiColombo (upcoming). \n\uf0b7 Can be seen from the earth only before sunrise and after sunset.  \nVenus \n\uf0b7 Known as Earth’s twin. \n\uf0b7 Longest rotation period of any planet. \n\uf0b7 Rotation period is longer than its orbital (revolution) period. \n\uf0b7 H

In [21]:
from langchain_community.llms import Ollama

llm = Ollama (
    model = "llama2"
)
llm


Ollama()

In [22]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    """Answer the following question based on the context below. Think step by step before providing a detailed answer.
    I will tip you if you answer correctly.
    Context: {context}
    Question: {input}"""
)

In [23]:
from langchain.chains.combine_documents import create_stuff_documents_chain

documents_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt
)

In [24]:
'''A retriever is an interface that returns documents given an unstructured query. It is more general than a vector store. A retriever does not need to be able to store documents, only to return (or retrieve) them. Vector stores can be used as the backbone of a retriever, but there are other types of retrievers as well.'''

retriever = db.as_retriever()



In [25]:
'''retriver chain is a chain that takes a query and returns a response based on the retrieved documents. It is a higher-level abstraction that combines the retriever and the document chain. The retriever chain can be used to answer questions based on the retrieved documents.'''

from langchain.chains import create_retrieval_chain
retriever_chain = create_retrieval_chain(
    retriever,
    documents_chain
)


In [30]:
response = retriever_chain.invoke({"input": "which planet Has the largest known volcano"})

In [31]:
response['answer']

'Based on the information provided in the context, the planet with the largest known volcano is Jupiter.'